In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
df = pd.read_csv("C:\\Users\\GK111\\Downloads\\audible_uncleaned.csv\\audible_uncleaned.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87489 entries, 0 to 87488
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   name         87489 non-null  object
 1   author       87489 non-null  object
 2   narrator     87489 non-null  object
 3   time         87489 non-null  object
 4   releasedate  87489 non-null  object
 5   language     87489 non-null  object
 6   stars        87489 non-null  object
 7   price        87489 non-null  object
dtypes: object(8)
memory usage: 5.3+ MB


In [4]:
df.sample(10)

,name,author,narrator,time,releasedate,language,stars,price
27210,Growing Up with Girl Parts,Writtenby:TatianaPérez,Narratedby:MariaLiatis,1 hr and 38 mins,10-11-16,English,Not rated yet,233.00
5049,"Listen, My Bridge Is So Cool!","Writtenby:NancyLoewen,CristianBernardini",Narratedby:anonymous,6 mins,24-03-22,English,Not rated yet,82.00
74594,Dangerous Games,Writtenby:SallySpencer,Narratedby:GarethArmstrong,6 hrs and 44 mins,27-03-14,English,Not rated yet,569.00
54660,Limoncello und die wahre Liebe,Writtenby:LotteRömer,Narratedby:VerenaWolfien,7 hrs and 25 mins,09-09-21,german,Not rated yet,267.00
49096,It's a Family Affair - We'll Settle It Ourselves,"Writtenby:AlexanderOstrovsky,GeorgeRapallNoyes...","Narratedby:JohnBurlinson,KristinaRothe,JoanDuK...",2 hrs and 45 mins,16-12-21,English,Not rated yet,335.00
21252,Epicuro [Epicurus],Writtenby:OnlineStudioProductions,Narratedby:uncredited,28 mins,23-01-18,spanish,Not rated yet,166.00
42939,Impresario of Castro Street,Writtenby:MarcHuestis,Narratedby:DanielHenning,10 hrs and 50 mins,01-10-19,English,Not rated yet,836.00
15760,第244回 新刊ラジオ第2部プレミアム,"Writtenby:矢島雅弘,石橋遊","Narratedby:矢島雅弘,石橋遊",17 mins,16-05-18,japanese,Not rated yet,139.00
58590,The Science of Getting Rich - Original Edition,Writtenby:WallaceD.Wattles,Narratedby:ChristaLewis,2 hrs and 23 mins,18-04-18,English,4.5 out of 5 stars25 ratings,152.00
72630,Bloodlust,Writtenby:HelenHarper,Narratedby:EmiliaWallace,8 hrs and 45 mins,28-06-21,german,Not rated yet,334.00


In [9]:
# Clean author
df['author'] = (
    df['author']
    .str.replace(r'Writtenby:', '', regex=True)   # remove "Writtenby:"
    .str.strip()                                 # remove extra spaces
)

# Clean narrator
df['narrator'] = (
    df['narrator']
    .str.replace(r'Narratedby:', '', regex=True) # remove "Narratedby:"
    .str.strip()
)

print(df['narrator'].sample(10))
print('###################')
print(df['author'].sample(10))

35233                      MatthewLloydDavies
35635                           DominikEisele
84406     BrianCox,HeathcoteWilliams,FullCast
18938                           GroverGardner
47003                    TheSt.CharlesPlayers
54204    RichardMitchley,GhizelaRowe,JakeUrry
36608                          AndrewParrella
85492                             ArturoLópez
79601                                 中西貴之,BJ
61546                           KirbyHeyborne
Name: narrator, dtype: object
###################
84919                              SharonCameron
33714                            DrW.ThomasBoyce
54505                                 FaithHogan
30879                            DanielMarkovitz
58107                              StuartE.Lucas
79717                                NicoleStott
39750    EvgeniBessonov,BairIrincheev-translator
4148                              CarolineArnold
58923                   BobSullivan,HughThompson
63158                              ThomasW.Young


In [6]:
#Clean rerease date
df['releasedate'] = pd.to_datetime(
    df['releasedate'],
    errors='coerce',              # invalid dates become NaT (null datetime)
    dayfirst=True                 # important: your sample is day-month-year
)
df['releasedate'].sample(10)

C:\Users\GK111\AppData\Local\Temp\ipykernel_5216\957617465.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['releasedate'] = pd.to_datetime(


11401   2021-11-16
24221   2010-06-02
4477    2022-04-04
25713   2015-10-06
40213   2021-05-20
9545    2018-11-27
70563   2021-06-11
17231   2019-10-31
29993   2016-09-22
14828   2019-09-10
Name: releasedate, dtype: datetime64[ns]

In [ ]:
# Function to convert strings like "2 hrs and 36 mins" to total minutes as int like 156
def convert_time_to_minutes(time_str):
    if pd.isna(time_str):   # Handle missing values
        return None
    
    # Extract hours and minutes using regex
    hours = re.search(r'(\d+)\s*hr', time_str)
    mins = re.search(r'(\d+)\s*min', time_str)
    
    total_minutes = 0
    if hours:
        total_minutes += int(hours.group(1)) * 60   # convert hrs to mins
    if mins:
        total_minutes += int(mins.group(1))         # add mins
    
    return total_minutes


In [13]:
# Clean stars and extract ratings into a new column leaving stars as float like 4.5 and ratings as int like 1234
def split_stars_and_ratings(text): # function to split stars and ratings
    if pd.isnull(text) or "not rated" in str(text).lower(): # Handle missing or "Not rated" values
        return pd.Series([None, None])# return two Nones
    
    pattern = re.compile(r'(\d+(?:\.\d+)?)\s*out of\s*5(?:\s+([\d,]+)\s*ratings)?', re.IGNORECASE) # Regex pattern to capture stars and optional ratings
    match = pattern.search(text) # search for the pattern

    if match:
        stars = float(match.group(1)) # convert stars to float
        ratings = int(match.group(2).replace(',', '')) if match.group(2) else None # convert ratings to int if present
        return pd.Series([stars, ratings]) # return as a Series for easy assignment to multiple columns
    
    return pd.Series([None, None]) # return two Nones if no match

# Apply the function to the 'stars' column and create two new columns 'stars_clean' and 'ratings'
df[['stars_clean', 'ratings']] = df['stars'].apply(split_stars_and_ratings) 


In [12]:
# Remove duplicates and reset index
df.drop_duplicates(inplace=True)
df.reset_index(drop=True, inplace=True)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87489 entries, 0 to 87488
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   name         87489 non-null  object        
 1   author       87489 non-null  object        
 2   narrator     87489 non-null  object        
 3   time         87489 non-null  object        
 4   releasedate  87489 non-null  datetime64[ns]
 5   language     87489 non-null  object        
 6   stars        87489 non-null  object        
 7   price        87489 non-null  object        
 8   stars_clean  15072 non-null  float64       
 9   ratings      0 non-null      float64       
dtypes: datetime64[ns](1), float64(2), object(7)
memory usage: 6.7+ MB
